In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [14]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-th8f9po0
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-th8f9po0
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15695 sha256=287bdc557b8cf83bef970a2e3d381d6c1a0daa9a61987b0cc60908fc59784434
  Stored in directory: /tmp/pip-ephem-wheel-cache-fvmgl3pm/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [3]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
#GPU 사용
device = torch.device("cuda:0")

In [6]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()


using cached model. /mnt/c/Users/Admin/Documents/kobert_project/.cache/kobert_v1.zip
using cached model. /mnt/c/Users/Admin/Documents/kobert_project/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


### Data 불러오기<br>
label에 대한 고민 <br>
<b>dataset 1(한국어 감정 정보가 포함된 단발성 대화 데이터셋)</b> : 공포 놀람 분노 슬픔 중립 행복 혐오 <br>
<b>dataset 2(감성 대화 말뭉치)</b> : 분노 슬픔 당황 기쁨 불안 상처 <br>
<br>
<b>5가지 분류</b> -> 분노혐오 / 중립 (행복 포함) / 슬픔 / 놀람 당황 / 공포 불안 <br>
<b>3가지 분류</b> -> 분노혐오 / 중립 (행복, 슬픔, 놀람, 당황) / 걱정(공포, 불안)

In [7]:
import pandas as pd 

dataset = pd.read_excel("/mnt/c/Users/Admin/documents/kobert_project/한국어_단발성_대화_데이터셋.xlsx")
dataset.isnull().sum()

Sentence          0
Emotion           0
Unnamed: 2    38594
Unnamed: 3    38594
Unnamed: 4    38594
공포            38587
5468.0        38587
dtype: int64

In [6]:
dataset.dropna(axis=1, inplace=True)
dataset.head()

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포


In [13]:
dataset.loc[dataset.Emotion == '놀람','Sentence'].tail(20)

11346          트럼프 70대 부인 40대.. 딸같은 ..
11347                      말이 많았을건데???
11348                         홍체인식????
11349                의외로 기능도 어려워 하시던데.
11350     세금 제대로 내는 직업에 종사하는 경우 드물다...
11351      저 여자애도 대단하지만 저 남자애가 더 대단하다.
11352                  우와 박보거어어어엄!!!!!
11353                 스마트폰이 앱을 사용치말라고?
11354            자사 제품 쓰지말라 한거면 말다했네..
11355                  설마 진짜 믿는건 아니겠지?
11356        스위스 정부의 법안이 아니야 ㅋㅋㅋㅋ ㅋㅋㅋㅋ
11357                          아이 큰일났네
11358           그만큼 수비 집중하게 햇던 아이버슨이라니
11359                 걷기 단추 눌렀더니 폭발...
11360                   더더군다나 그게 약소국..
11361                시대가 어느시대인데 사람이 뽑냐
11362                   진심 내 인생 드라마임..
11363                       대단하다... 진짜
11364                   헐 김종민이가 대상급인가?
11365                    7로 나온다든데 아니었나
Name: Sentence, dtype: object

In [9]:
dataset['Emotion'].value_counts()

행복    6037
놀람    5898
분노    5665
공포    5468
혐오    5429
슬픔    5267
중립    4830
Name: Emotion, dtype: int64

In [8]:
dataset = dataset.loc[dataset.Emotion != '놀람', :]
dataset = dataset.loc[dataset.Emotion != '슬픔', :]
dataset['Emotion'].value_counts()

행복    6037
분노    5665
공포    5468
혐오    5429
중립    4830
Name: Emotion, dtype: int64

In [9]:
mapping_emotions = {
  "분노" : 0,
  "혐오" : 0,
  "중립" : 1,
  "기쁨" : 1,
  "행복" : 1,
  "공포" : 1,
  "불안" : 1,
  }

dataset["Label"] = dataset["Emotion"].map(mapping_emotions)
dataset.head()

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468.0,Label
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0,1
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0,1
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0,1
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0,1
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0,1


In [10]:
import pandas as pd 
emotion_data_1 = pd.read_excel("/mnt/c/Users/Admin/documents/kobert_project/감성대화말뭉치(최종데이터)_Training.xlsx")
emotion_data_2 = pd.read_excel("/mnt/c/Users/Admin/documents/kobert_project/감성대화말뭉치(최종데이터)_Validation.xlsx")
emotion_data = pd.concat([emotion_data_1, emotion_data_2], axis=0)
emotion_data.isnull().sum()

Unnamed: 0        0
연령                0
성별                0
상황키워드             0
신체질환              0
감정_대분류            0
감정_소분류            0
사람문장1             0
시스템문장1            0
사람문장2             0
시스템문장2            0
사람문장3         10890
시스템문장3        10890
dtype: int64

In [21]:
emotion_data.감정_대분류.value_counts()

불안    10433
분노    10417
상처    10150
슬픔    10128
당황     9804
기쁨     7339
Name: 감정_대분류, dtype: int64

In [10]:
emotion_data.loc[emotion_data.감정_대분류 == '당황',['사람문장1','사람문장2','사람문장3']].tail(20)

,사람문장1,사람문장2,사람문장3
6564,우리 아파트 동대표들이 작당하고 돈 장난을 친다니 혐오스러운 놈들이야.,술집에서 자기들끼리 싸우고 나서 한 놈이 약간 까발린 모양이야. 대한민국 아파트가 ...,NaN
6565,나 오늘 회사에서 넘어져서 많이 부끄러웠어.,다친 곳은 전혀 없는데 사람 많은 곳에서 넘어져서 그냥 많이 부끄러웠어.,NaN
6573,오늘 애들이 갑자기 나만 놔두고 밥을 먹으러 가서 당황했어.,물어볼 용기가 안 나서 못 했어.,NaN
6580,여자친구랑 뷔페를 갔는데 좀 당황스러운 일이 있었어.,여자친구가 처음에 디저트를 접시에 마구 퍼오더라고. 근데 매우 많은 양이었어.,인증샷만 찍고 거의 먹지 않더라고. 음식을 함부로 하는 모습에 실망했어.
6583,부모님이 동생만 챙기셔서 너무 힘들어.,집에 들어가는 것이 너무 싫고 힘들어.,NaN
6584,가족과 사이가 좋지 않아서 내가 한심한 기분이 들어.,글쎄 말하고 싶지 않아.,NaN
6594,이게 갑자기 무슨 일이지?,친구에게서 갑자기 다시는 연락하지 말라는 문자를 받았어. 이런 통보는 또 처음이야.,NaN
6595,상사한테 한 소리 들었어. 당황스러워.,동료가 실수한 일을 나한테 얘기하시네!,NaN
6599,성격이 내성적이라서 사람들 앞에 나서기가 부끄러워.,부끄러워서 많은 사람 앞에서 대화를 길게 못 하겠어. 도움이 되는 책을 사서 읽기 ...,NaN
6600,회사 일로 발표를 할 땐 늘 상사에게 혼나게 돼.,사람들 앞에서 얘기하는 게 부끄러워서 늘 발표를 망쳐. 어떻게 해야 할지 모르겠어.,NaN


In [11]:
target_column = ['감정_대분류', '사람문장1', '사람문장2', '사람문장3']
print(f'가공 전 데이터 개수 : {len(emotion_data)}')
emotion_data = emotion_data.loc[emotion_data.감정_대분류 != '슬픔', target_column]
emotion_data = emotion_data.loc[emotion_data.감정_대분류 != '상처', target_column]
emotion_data = emotion_data.loc[emotion_data.감정_대분류 != '당황', target_column]


emotion_data_mod = emotion_data.dropna(axis=0).reset_index(drop=True)
print(f'가공 후 데이터 개수 : {len(emotion_data_mod)}')
print(f'가공 후 null값 개수 \n{emotion_data_mod.isnull().sum()}')


emotion_data_mod.columns = ['Emotion', 'Sent1', 'Sent2', 'Sent3']
emotion_data_mod

가공 전 데이터 개수 : 58271
가공 후 데이터 개수 : 22514
가공 후 null값 개수 
감정_대분류    0
사람문장1     0
사람문장2     0
사람문장3     0
dtype: int64


,Emotion,Sent1,Sent2,Sent3
0,기쁨,아프긴 해도 혼자 사니까 다른 사람 신경 쓰지 않아도 되고 모은 돈으로 살 수 있어...,혼자 살아서 돈을 넉넉하게 모을 수 있었어. 병원비가 꽤 나가더라고.,내가 좋아하는 취미 생활을 찾는 것이 가장 중요한 것 같아.
1,기쁨,경제적으로 여유 있어 아픈 것만 빼면 삶이 너무 즐거워.,어릴 때 열심히 일해서 벌어 두길 정말 잘한 것 같아.,건강을 위해 운동을 꾸준히 하며 관리하고 있어.
2,기쁨,건강 검진을 받아도 경제적으로 여유로워 마음이 편해.,맞아. 검사비가 많이 나오니까 전에는 그런 점들이 걱정이었어.,재테크에 성공해서 경제적인 여유가 생겼어.
3,기쁨,만성 질환을 앓고 있어도 경제적으로 여유로워 약값 걱정이 안 돼.,그래서 약을 꾸준히 먹고 있어. 만성 질환엔 꾸준히 약을 먹는 게 중요하더라고.,당연하지. 계속 잘 관리해서 건강하게 오래오래 살 거야.
4,기쁨,아들이 병원 갈 때 함께 가주고 병원비도 내줘서 너무 든든해.,아들이 병원 예약이 있는 날마다 항상 함께 가 줘. 병원비도 부담될 텐데 전혀 내색...,아들을 위해서 더욱 건강하게 관리해야지.
...,...,...,...,...
22509,기쁨,출산을 무사히 마치고 퇴원했어. 많은 사람이 축하해줬는데 그분들께 너무 감사해.,출산 직후에는 정신이 없었는데 퇴원하고 여유가 생기니 주변 사람들에게 감사하는 마음...,그렇지. 이제 힘내서 사람들에게 감사 인사를 전하는 연락을 할 거야.
22510,분노,나 정말 화가 나 죽겠어.,친구가 약속 시각에 자꾸 늦어.,말해봐도 소용이 없어.
22511,기쁨,내 나이가 되면 원망도 줄어들고 고마운 마음이 더 많아지는 거 같아.,키워주신 부모님 묵묵히 가정을 지켜준 남편에게 고마워.,부모님께는 자주 찾아뵙는 게 우선일 거고 남편에겐 힘낼 수 있는 말을 자주 해 줄 ...
22512,불안,나 요즘 친구들이 나를 은근히 피하고 장난도 더 심하게 치는 것 같아서 혹시 나를 ...,친구들이 나를 왜 피하는지도 모르겠어.,좋아하는 노래를 한 곡 듣고 기분이 나아진 상태에서 친구들과 대화를 하면 좋을 것 같아.


In [56]:
emotion_data_mod.Emotion.value_counts()

분노    8576
불안    8454
기쁨    5484
Name: Emotion, dtype: int64

In [12]:
mapping_emotions = {
  "분노" : 0,
  "혐오" : 0,
  "중립" : 1,
  "기쁨" : 1,
  "행복" : 1,
  "공포" : 1,
  "불안" : 1,
  }
emotion_data_mod["Label"] = emotion_data_mod["Emotion"].map(mapping_emotions)

In [13]:
data_list_one = []
for i in range(1,4):
  for q, label in zip(emotion_data_mod[f'Sent{i}'], emotion_data_mod['Label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list_one.append(data)

print(f"변환 정보의 길이 : {len(data_list_one)}\n원본 데이터의 길이 : {len(emotion_data_mod)}")
if len(data_list_one) == len(emotion_data_mod)*3:
  print("데이터 변환이 완료되었습니다.")

변환 정보의 길이 : 67542
원본 데이터의 길이 : 22514
데이터 변환이 완료되었습니다.


In [14]:
data_list_two = []
for q, label in zip(dataset['Sentence'], dataset['Label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list_two.append(data)

data_list = data_list_one + data_list_two

In [15]:
from sklearn.model_selection import train_test_split

train_data, test_data =  train_test_split(data_list, test_size=0.25, random_state=0)
print(len(train_data))
print(len(test_data))

71228
23743


In [16]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [17]:
# Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [18]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_data, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_data, 0, 1, tok, max_len, True, False)

using cached model. /mnt/c/Users/Admin/Documents/kobert_project/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [19]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [20]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [21]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

KeyboardInterrupt: 

In [23]:
for e in range(num_epochs//2):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    
    if (e+1) % 2 == 1:
        torch.save({'epochs' : num_epochs+1,
                    'model' : model.state_dict(),
                    'optimizer' : optimizer.state_dict()
                    }, f'/mnt/c/Users/Admin/documents/kobert_project/checkpoint/checkpoint_ep_{e+1}_{train_acc/(batch_id+1)}.pt')

/home/ryud904/miniconda3/envs/kobert/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/2226 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7449871301651001 train acc 0.4375
epoch 1 batch id 201 loss 0.7316056489944458 train acc 0.5309390547263682
epoch 1 batch id 401 loss 0.4899197816848755 train acc 0.5912562344139651
epoch 1 batch id 601 loss 0.5494669079780579 train acc 0.6452787021630616
epoch 1 batch id 801 loss 0.39243295788764954 train acc 0.670099875156055
epoch 1 batch id 1001 loss 0.2891656160354614 train acc 0.6878434065934066
epoch 1 batch id 1201 loss 0.2626956105232239 train acc 0.6985064529558701
epoch 1 batch id 1401 loss 0.4477037191390991 train acc 0.707151142041399
epoch 1 batch id 1601 loss 0.3023909330368042 train acc 0.7143386945658963
epoch 1 batch id 1801 loss 0.49119502305984497 train acc 0.7186979455857857
epoch 1 batch id 2001 loss 0.44536805152893066 train acc 0.7223419540229885
epoch 1 batch id 2201 loss 0.5472137331962585 train acc 0.7253947069513857
epoch 1 train acc 0.7258254716981132


/home/ryud904/miniconda3/envs/kobert/lib/python3.7/site-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/742 [00:00<?, ?it/s]

...

epoch 4 test acc 0.7736990261716373


  0%|          | 0/2226 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.2936319410800934 train acc 0.875
epoch 5 batch id 201 loss 0.18882468342781067 train acc 0.8970771144278606
epoch 5 batch id 401 loss 0.30801352858543396 train acc 0.9006390274314214
epoch 5 batch id 601 loss 0.2000262588262558 train acc 0.9057300332778702
epoch 5 batch id 801 loss 0.09665460139513016 train acc 0.9079275905118602
epoch 5 batch id 1001 loss 0.05116917937994003 train acc 0.9105894105894106
epoch 5 batch id 1201 loss 0.026855556294322014 train acc 0.9127289758534555
epoch 5 batch id 1401 loss 0.0471235066652298 train acc 0.9153283369022127
epoch 5 batch id 1601 loss 0.14082352817058563 train acc 0.9155605871330419
epoch 5 batch id 1801 loss 0.24193720519542694 train acc 0.9167649916712938
epoch 5 batch id 2001 loss 0.189158633351326 train acc 0.9168228385807097
epoch 5 batch id 2201 loss 0.3069063425064087 train acc 0.918488755111313
epoch 5 train acc 0.918497705686048


  0%|          | 0/742 [00:00<?, ?it/s]

epoch 5 test acc 0.7771973632727589


In [26]:
# 학습 재개(gpu 메모리로 인해 나눠서 학습을 진행합니다.)

checkpoint = torch.load('./checkpoint/checkpoint_ep_4_2.650293608009241.pt')
model = BERTClassifier(bertmodel,  dr_rate=0.5)
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

<All keys matched successfully>

In [ ]:


for e in range(num_epochs//2, num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    
    if (e+1) % 2 == 0:
        torch.save({'epochs' : num_epochs+1,
                    'model' : model.state_dict(),
                    'optimizer' : optimizer.state_dict()
                    }, f'/mnt/c/Users/Admin/documents/kobert_project/checkpoint/checkpoint_ep_{e+1}_{train_acc/(batch_id+1)}.pt')

In [37]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("분노 및 혐오가")
            elif np.argmax(logits) == 1:
                test_eval.append("일반적 감정이")
            
        print(">> 입력하신 내용에서 " + test_eval[0] + " 감지됩니다.")

using cached model. /mnt/c/Users/Admin/Documents/kobert_project/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [46]:
#질문 무한반복하기! 0 입력시 종료

for i in range(5) :
  sentence = input("하고싶은 말을 입력해주세요 : ")
  print(sentence)
  predict(sentence)
  print("\n")

사랑해
>> 입력하신 내용에서 일반적 감정이 감지됩니다.


아 그게 아니잖아....
>> 입력하신 내용에서 일반적 감정이 감지됩니다.


아 그게 아니라;
>> 입력하신 내용에서 일반적 감정이 감지됩니다.


아니아니 그게 아니라
>> 입력하신 내용에서 분노 및 혐오가 감지됩니다.


짜증나....
>> 입력하신 내용에서 분노 및 혐오가 감지됩니다.


